## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

import requests
import zipfile
import io

## Downloading and reading datasets

### If you prefer downloading it manually, World Development Indicators (WDI) database can be found [here](https://datacatalog.worldbank.org/search/dataset/0037712)

In [2]:
# Direct URL to the World Bank's ZIP file
zip_url = 'https://datacatalogfiles.worldbank.org/ddh-published/0037712/DR0095335/WDI_CSV_2025_07_02.zip'

print("Starting the download of the World Bank dataset...")

try:
    # Make the request to the URL and store the response
    response = requests.get(zip_url)
    response.raise_for_status()  # Raise an error if the download fails

    # Store the zip content in an in-memory BytesIO object
    # This object behaves like a file and can be used by the next cell
    zip_content_in_memory = io.BytesIO(response.content)

    print("Download completed successfully!")
    print("The ZIP file content is stored in the 'zip_content_in_memory' variable.")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

Starting the download of the World Bank dataset...
Download completed successfully!
The ZIP file content is stored in the 'zip_content_in_memory' variable.


In [3]:
csv_data_filename = 'WDICSV.csv'
csv_country_filename = 'WDICountry.csv'

try:
    # Open the ZIP file that is in memory
    with zipfile.ZipFile(zip_content_in_memory, 'r') as zip_ref:

        print(f"Reading '{csv_data_filename}'...")
        # Open and read the first CSV file into the df_indicators dataframe
        with zip_ref.open(csv_data_filename) as file:
            df_indicators = pd.read_csv(file)

        print(f"Reading '{csv_country_filename}'...")
        # Open and read the second CSV file into the df_countries dataframe
        with zip_ref.open(csv_country_filename) as file:
            df_countries = pd.read_csv(file)

        print("\nDataframes loaded successfully!")

except NameError:
    print("Error: The 'zip_content_in_memory' variable was not found.")
    print("Please, run Cell 1 first to download the file.")
except KeyError as e:
    print(f"\nError: One of the CSV files was not found in the ZIP archive: {e}")
    # If a file is not found, this part helps with debugging
    with zipfile.ZipFile(zip_content_in_memory, 'r') as zip_ref:
        print("Available files in the ZIP:", zip_ref.namelist())

Reading 'WDICSV.csv'...
Reading 'WDICountry.csv'...

Dataframes loaded successfully!


In [4]:
df = df_indicators.merge(df_countries[['Country Code', 'Region']], how='left', on='Country Code')

In [5]:
df.shape

(403256, 70)

## Example and feature selection

### Selecting 16 indicators

In [6]:
indicators = {
    # Demography
    "SP.POP.TOTL": "population_total",
    "SP.URB.TOTL.IN.ZS": "urban_population_percent",
    "SP.POP.65UP.TO.ZS": "population_65_plus_percent",
    "SP.DYN.TFRT.IN": "fertility_rate_total",
    "SP.DYN.LE00.IN": "life_expectancy_at_birth",

    # Economy
    "NY.GDP.PCAP.CD": "gdp_per_capita_usd",
    "NY.GDP.MKTP.KD.ZG": "gdp_growth_annual_percent",
    "NV.AGR.TOTL.ZS": "agriculture_value_added_percent_gdp",
    "NV.IND.TOTL.ZS": "industry_value_added_percent_gdp",

    # Infrastructure & Technology
    "EG.ELC.ACCS.ZS": "access_to_electricity_percent",
    "IT.CEL.SETS.P2": "mobile_cellular_subscriptions_per_100_people",

    # Health & Environment
    "SH.XPD.CHEX.GD.ZS": "health_expenditure_pct_gdp",
    "SP.DYN.IMRT.IN": "child_mortality_rate",
    "EN.GHG.CO2.PC.CE.AR5": "co2_emissions_per_capita",

    # Governance & Stability
    "PV.EST": "stability_of_government_estimate",
    "CC.EST": "corruption_perception_estimate"
}

df_filtered = df[df['Indicator Code'].isin(indicators.keys())].copy()

### Changing indicator names

In [7]:
df_filtered["Indicator Name"] = df_filtered["Indicator Code"].map(indicators)

### Removing lines related to regions, and not countries

In [8]:
df_filtered = df_filtered[df_filtered["Region"].notna()].reset_index(drop=True)

### Selecting years from 21st century

In [9]:
years_to_remove = [str(year) for year in range(1960, 2001)]
df_21century = df_filtered.drop(columns=years_to_remove)

In [10]:
df_21century.shape

(3472, 29)

## Year selection

### Percentage of NaN values for every Indicator Code in each Year

In [11]:
year_columns = [col for col in df_21century.columns if col.isdigit()]

nan_percentage_by_indicator_year = df_21century.groupby('Indicator Code')[year_columns].apply(
    lambda x: x.isnull().sum() / len(x) * 100
)

display(nan_percentage_by_indicator_year)

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Indicator Code,,,,,,,,,,,,,,,,,,,,,
CC.EST,100.000000,13.824885,13.824885,8.755760,8.294931,8.294931,7.834101,7.373272,6.912442,6.451613,...,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,100.000000
EG.ELC.ACCS.ZS,2.764977,2.304147,2.304147,2.304147,2.304147,2.304147,1.382488,1.382488,0.921659,0.921659,...,0.921659,0.921659,0.921659,0.921659,0.921659,0.921659,0.921659,0.921659,0.921659,100.000000
EN.GHG.CO2.PC.CE.AR5,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,...,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,6.451613,100.000000
IT.CEL.SETS.P2,5.990783,5.069124,5.069124,4.608295,5.990783,7.373272,5.990783,6.912442,5.990783,4.608295,...,4.608295,6.912442,6.451613,11.981567,4.147465,4.608295,4.147465,3.686636,25.345622,100.000000
NV.AGR.TOTL.ZS,14.746544,13.824885,12.903226,12.903226,12.903226,11.059908,11.059908,9.677419,9.216590,9.216590,...,7.834101,7.834101,7.834101,8.294931,8.294931,9.216590,9.677419,10.138249,14.746544,29.032258
NV.IND.TOTL.ZS,16.589862,15.668203,14.285714,13.824885,13.824885,11.981567,11.520737,10.138249,9.677419,8.755760,...,5.990783,6.451613,6.451613,6.912442,6.912442,7.834101,8.294931,9.216590,13.824885,29.032258
NY.GDP.MKTP.KD.ZG,7.834101,7.834101,5.990783,5.990783,5.990783,5.990783,5.529954,5.069124,3.686636,3.225806,...,3.225806,3.686636,3.686636,3.225806,3.686636,3.686636,3.686636,4.147465,7.373272,15.207373
NY.GDP.PCAP.CD,5.529954,3.686636,3.686636,3.686636,3.686636,3.225806,3.225806,2.764977,1.843318,1.843318,...,2.764977,3.225806,3.225806,3.225806,2.764977,3.225806,3.225806,3.686636,7.373272,15.207373
PV.EST,100.000000,13.824885,9.216590,7.373272,7.373272,6.912442,6.912442,6.451613,5.990783,5.990783,...,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,5.529954,100.000000


### Selecting only 2022

In [12]:
ANO_ANALISE = '2022'

df_pivot_subset = df_21century[['Country Name', 'Country Code', 'Region', 'Indicator Code', ANO_ANALISE]]

df_pivot = df_pivot_subset.pivot_table(
    index=['Country Name', 'Country Code', 'Region'],
    columns='Indicator Code',
    values=ANO_ANALISE
).reset_index()

df_2022 = df_pivot.rename(columns=indicators)

## Dealing with NaN values

### Countries with NaN values

In [13]:
dfna = df_2022[df_2022.isna().any(axis=1)]
na_counts = dfna.isna().sum(axis=1)

# Create a mapping from indicator names to numbers
indicator_mapping = {name: i for i, name in enumerate(indicators.values())}

na_indicators_list = []
for index, row in dfna.iterrows():
    na_cols = row[row.isna()].index.tolist()
    # Map indicator names to numbers
    na_indicator_numbers = [indicator_mapping[col] for col in na_cols if col in indicator_mapping]
    na_indicators_list.append(na_indicator_numbers)

na_countries = pd.DataFrame({
    'Country Name': dfna['Country Name'],
    'Population Total': dfna['population_total'],
    'NA Count': na_counts,
    'NA Indicators': na_indicators_list
}).sort_values(by='NA Count', ascending=False)

In [14]:
indicator_mapping

{'population_total': 0,
 'urban_population_percent': 1,
 'population_65_plus_percent': 2,
 'fertility_rate_total': 3,
 'life_expectancy_at_birth': 4,
 'gdp_per_capita_usd': 5,
 'gdp_growth_annual_percent': 6,
 'agriculture_value_added_percent_gdp': 7,
 'industry_value_added_percent_gdp': 8,
 'access_to_electricity_percent': 9,
 'mobile_cellular_subscriptions_per_100_people': 10,
 'health_expenditure_pct_gdp': 11,
 'child_mortality_rate': 12,
 'co2_emissions_per_capita': 13,
 'stability_of_government_estimate': 14,
 'corruption_perception_estimate': 15}

In [15]:
na_countries

,Country Name,Population Total,NA Count,NA Indicators
183,St. Martin (French part),28870.0,11,"[15, 13, 10, 7, 8, 6, 5, 14, 11, 12, 1]"
75,Gibraltar,37609.0,8,"[15, 7, 8, 6, 5, 14, 11, 12]"
146,Northern Mariana Islands,46078.0,7,"[15, 10, 7, 8, 14, 11, 12]"
172,Sint Maarten (Dutch part),42139.0,7,"[15, 13, 7, 8, 14, 11, 12]"
27,British Virgin Islands,38319.0,7,"[15, 7, 8, 6, 5, 14, 11]"
94,Isle of Man,84132.0,6,"[15, 13, 10, 14, 11, 12]"
69,French Polynesia,280378.0,6,"[15, 7, 8, 14, 11, 12]"
140,New Caledonia,287123.0,6,"[15, 7, 8, 14, 11, 12]"
3,American Samoa,48342.0,6,"[9, 10, 7, 8, 11, 12]"
39,Channel Islands,167215.0,6,"[15, 13, 10, 14, 11, 12]"


### World map vision

In [16]:
year_columns = [col for col in df_21century.columns if col.isdigit()]

nan_count_by_country_year = df_21century.groupby('Country Name')[year_columns].apply(
    lambda x: x.isnull().sum()
)

nan_2022 = nan_count_by_country_year["2022"].reset_index()

nan_2022 = df_21century[["Country Name", "Country Code"]].drop_duplicates().merge(
    nan_2022, on="Country Name", how="right"
)

fig = px.choropleth(
    nan_2022,
    locations="Country Code",
    color="2022",
    hover_name="Country Name",
    color_continuous_scale="Reds",
    title="Absolute NaN values for each country in 2022"
)

fig.show()

### Remove countries with 5 or more NaN values

In [17]:
countries_to_remove = na_countries[na_countries["NA Count"] > 4]
countries_to_remove_list = countries_to_remove["Country Name"].tolist()
df_final = df_2022[~df_2022["Country Name"].isin(countries_to_remove_list)].reset_index(drop=True)

### Imputing values in the rest of the countries

In [ ]:
cols_identification = ['Country Name', 'Country Code', 'Region']
cols_indicators = [col for col in df_2022.columns if col not in cols_identification]

df_identification = df_2022[cols_identification]
df_indicators = df_2022[cols_indicators]

scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_indicators)
df_scaled = pd.DataFrame(data_scaled, columns=cols_indicators, index=df_indicators.index)

imputer = KNNImputer(n_neighbors=5)
data_imputed_scaled = imputer.fit_transform(df_scaled)
df_imputed_scaled = pd.DataFrame(data_imputed_scaled, columns=cols_indicators, index=df_indicators.index)

data_imputed_original = scaler.inverse_transform(df_imputed_scaled)
df_imputed_final = pd.DataFrame(data_imputed_original, columns=cols_indicators, index=df_indicators.index)
df_final = pd.concat([df_identification, df_imputed_final], axis=1)

### Last Check for Null Values

In [19]:
df_final.isnull().sum()

Country Name                                    0
Country Code                                    0
Region                                          0
corruption_perception_estimate                  0
access_to_electricity_percent                   0
co2_emissions_per_capita                        0
mobile_cellular_subscriptions_per_100_people    0
agriculture_value_added_percent_gdp             0
industry_value_added_percent_gdp                0
gdp_growth_annual_percent                       0
gdp_per_capita_usd                              0
stability_of_government_estimate                0
health_expenditure_pct_gdp                      0
child_mortality_rate                            0
life_expectancy_at_birth                        0
fertility_rate_total                            0
population_65_plus_percent                      0
population_total                                0
urban_population_percent                        0
dtype: int64

### Correcting some strange values calculated by KNNImputer

In [21]:
df_final.loc[df_final["Country Name"] == "Venezuela, RB", "gdp_per_capita_usd"] = 3306.912
df_final.loc[df_final["Country Name"] == "Cuba", "gdp_per_capita_usd"] = 10000.00

## Saving the preprocessed dataset

In [22]:
final_path = 'WDI2022.csv'
df_final.to_csv(final_path, index=False)